In [1]:
from sagemaker import get_execution_role
from sagemaker.session import Session

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = 'car-classifier-us-east-2/python_readable_data/stanford_cars_dataset/SGM_data_and_models'

#IAM execution role that gives SageMaker access to resources in your AWS account.
role = get_execution_role()

Define Model

In [2]:
from sagemaker.tensorflow import TensorFlow

car_classifier = TensorFlow(entry_point='CNN_TF_script.py',
                            role=role,
                            train_volume_size = 30,
                            train_max_run = 600, #seconds to run before terminating
                            py_version = 'py3',
                            train_instance_count=1,
                            train_instance_type='ml.m4.xlarge',                           
                            framework_version = '1.12.0',
                            distributions={'parameter_server': {'enabled': True}}
                           )

Train

In [ ]:
%%time
import boto3

# use the region-specific sample data bucket
train_data_location = 's3://{}/data/train.csv'.format(bucket)
test_data_location = 's3://{}/data/test.csv'.format(bucket)

car_classifier.fit({'train' : train_data_location,
                   'test' : test_data_location
                  })

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2019-03-21-18-54-33-122


2019-03-21 18:54:33 Starting - Starting the training job...
2019-03-21 18:54:34 Starting - Launching requested ML instances......
2019-03-21 18:55:36 Starting - Preparing the instances for training...
2019-03-21 18:56:33 Downloading - Downloading input data..

Deploy from object

In [5]:
%%time
car_predictor = car_classifier.deploy(initial_instance_count=1,
                                      instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091


---------------------------------------------------!CPU times: user 274 ms, sys: 18.6 ms, total: 292 ms
Wall time: 4min 18s


Deploy from file

In [23]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://sagemaker-us-east-2-050922845674/' +
              'sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091/output/model.tar.gz', role=role)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-serving-2019-03-20-22-59-50-675
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-serving-2019-03-20-22-59-50-675


---------------------------------------------------------------!

Predict

In [12]:
import numpy as np

head_line = np.loadtxt('/home/ec2-user/SageMaker/train_head.csv', delimiter=',')

head_line = head_line[:, :-2].reshape((30, 200, 200, 1))

In [22]:
#object
car_predictor.predict(head_line[0,:])

{'predictions': [{'probabilities': [1.0, 0.0],
   'logits': [120.136, -124.957],
   'classes': 0}]}

In [24]:
#file
car_predictor.predict(head_line[0,:])

{'predictions': [{'probabilities': [1.0, 0.0],
   'logits': [120.136, -124.957],
   'classes': 0}]}

Clean up

In [1]:
import boto3

client = boto3.client('sagemaker')

In [5]:
client.delete_endpoint(EndpointName = 'sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091')

{'ResponseMetadata': {'RequestId': 'fd14ade2-dfe2-4589-bf3f-d95f44b3746f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd14ade2-dfe2-4589-bf3f-d95f44b3746f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 20 Mar 2019 23:09:15 GMT'},
  'RetryAttempts': 0}}

In [6]:
client.delete_model(ModelName = 'sagemaker-tensorflow-serving-2019-03-20-22-59-50-675')

{'ResponseMetadata': {'RequestId': '0de5a353-b9ec-446a-9039-0a813fbda7bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0de5a353-b9ec-446a-9039-0a813fbda7bb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 20 Mar 2019 23:09:33 GMT'},
  'RetryAttempts': 0}}